In [61]:
import findspark
import pyspark
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

In [62]:
# Inicializa o SparkSession
spark = SparkSession.builder \
    .appName("ExemploSpark") \
    .getOrCreate()

# Verifica a versão do Spark para confirmar a configuração
print("Versão do Spark:", spark.version)


Versão do Spark: 3.5.2


24/08/26 13:49:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [63]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
import findspark
findspark.init()

In [64]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Criar uma sessão Spark
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .getOrCreate()

# Verificar se a sessão foi criada corretamente
print(spark)

arquivo = "/home/wermelingerv/Documentos/GitHub/SoulCode/Bootcamp - ANALISE DE DADOS/Banco de Dados_base_suja.csv"

df_base_suja = spark\
    .read.format("csv")\
    .option("inferSchema", "True")\
    .option("header", "True")\
    .csv(arquivo)
df_base_suja.show()

24/08/26 13:49:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+--------------+-----+----------+-------+-------------------+-----------------+
| ID|          Nome|Idade|      Área|Salário|Data de Contratação|Status de Emprego|
+---+--------------+-----+----------+-------+-------------------+-----------------+
|  1|    João Silva|   29|Financeiro|   5500|         01/02/2020|            Ativo|
|  2|   Maria Souza|   30|        RH|   4800|         15-03-2019|            ativo|
|  3|Carlos Pereira| NULL| Finaceiro|   6200|         2020/04/01|            Ativo|
|  4|     Ana Clara|   27| Marketing|   4800|         12/06/2018|            Ativo|
|  5|  Fabio Santos|   31|      NULL|   5500|         20-11-2017|            ativo|
|  6|   Sandra Lima|   28|        RH|   NULL|         05-05-2020|            Ativo|
|  7|    José Alves|   34| Marketing|   5400|         2018/02/01|          inativo|
|  8| Luciana Costa|   30|Financeiro|R$ 5200|         01.01.2019|            Ativo|
|  9| Paulo Ricardo| NULL| Finaceiro|   6100|         12/12/2020|          I

In [65]:
#1. Análise de Consistência de Dados: 
# Identificar e corrigir inconsistências nos valores de Área e Status de Emprego(e.g.,diferenças de maiúsculas/minúsculas).

# Renomear a coluna "Área" para "Area" e converter as colunas para minúsculas em uma única linha
status_area = df_base_suja.withColumnRenamed("Área", "Area") \
    .selectExpr("lower(Area) as Area", "lower(`Status de Emprego`) as `Status de Emprego`")
# Mostrar o resultado
status_area.show()

# 2. Limpeza de Dados Faltantes: Detectar e tratar os valores ausentes na coluna "Idade" e "Área".
# Contar valores ausentes na coluna "Idade"
df_base_suja.select(count(when(col("Idade").isNull() | isnan(col("Idade")), "Idade")).alias("Idade Missing"))
# Contar valores ausentes na coluna "Área"
df_base_suja.select(count(when(col("Área").isNull() | isnan(col("Área")), "Área")).alias("Área Missing"))
# Remover linhas onde "Idade" ou "Área" tem valores ausentes
df_cleaned = df_base_suja.dropna(subset=["Idade", "Área"])
# Verificar se os valores ausentes foram removidos
df_cleaned.show()


+----------+-----------------+
|      Area|Status de Emprego|
+----------+-----------------+
|financeiro|            ativo|
|        rh|            ativo|
| finaceiro|            ativo|
| marketing|            ativo|
|      NULL|            ativo|
|        rh|            ativo|
| marketing|          inativo|
|financeiro|            ativo|
| finaceiro|          inativo|
|        rh|            ativo|
|financeiro|            ativo|
+----------+-----------------+

+---+-------------+-----+----------+-------+-------------------+-----------------+
| ID|         Nome|Idade|      Área|Salário|Data de Contratação|Status de Emprego|
+---+-------------+-----+----------+-------+-------------------+-----------------+
|  1|   João Silva|   29|Financeiro|   5500|         01/02/2020|            Ativo|
|  2|  Maria Souza|   30|        RH|   4800|         15-03-2019|            ativo|
|  4|    Ana Clara|   27| Marketing|   4800|         12/06/2018|            Ativo|
|  6|  Sandra Lima|   28|        RH|

In [68]:
# 1. Renomear a coluna "Área" para "Area" e converter as colunas para minúsculas, incluindo "Idade"
df_cleaned = df_base_suja.withColumnRenamed("Área", "Area").selectExpr("lower(Area) as Area", "lower(`Status de Emprego`) as `Status de Emprego`", "Idade")

# 2. Detectar e tratar os valores ausentes na coluna "Idade" e "Area"
df_cleaned = df_cleaned.dropna(subset=["Idade", "Area"])

# 3. Analisar a distribuição da idade dos funcionários por departamento
df_age_distribution = df_cleaned.groupBy("Area").agg(
        avg("Idade").alias("Média de Idade"),
        stddev("Idade").alias("Desvio Padrão de Idade"),
        count("Idade").alias("Contagem de Funcionários"))

# Mostrar o resultado da análise de distribuição de idade por departamento
df_age_distribution.show()

+----------+------------------+----------------------+------------------------+
|      Area|    Média de Idade|Desvio Padrão de Idade|Contagem de Funcionários|
+----------+------------------+----------------------+------------------------+
|financeiro|29.333333333333332|    0.5773502691896258|                       3|
| marketing|              30.5|     4.949747468305833|                       2|
|        rh|              29.0|                   1.0|                       3|
+----------+------------------+----------------------+------------------------+



In [74]:
# 1. Renomear a coluna "Área" para "Area" e converter as colunas para minúsculas
df_cleaned = df_base_suja.withColumnRenamed("Área", "Area").selectExpr("lower(Area) as Area", "lower(`Salário`) as Salario")

# 2. Detectar e tratar os valores ausentes na coluna "Salário" e "Area"
df_cleaned = df_cleaned.dropna(subset=["Salario", "Area"])

# 4. Calcular a média, mediana e desvio padrão dos salários por departamento
df_salary_stats = df_cleaned.groupBy("Area").agg(
        avg("Salario").alias("Média de Salário"),
        stddev("Salario").alias("Desvio Padrão de Salário"),
        percentile_approx("Salario", 0.5).alias("Mediana de Salário"))

# Mostrar o resultado da análise de salário por departamento
df_salary_stats.show()


+----------+----------------+------------------------+------------------+
|      Area|Média de Salário|Desvio Padrão de Salário|Mediana de Salário|
+----------+----------------+------------------------+------------------+
|financeiro|          5500.0|                     0.0|            5500.0|
| marketing|          5100.0|      424.26406871192853|            4800.0|
|        rh|          4800.0|                     0.0|            4800.0|
| finaceiro|          6150.0|       70.71067811865476|            6100.0|
+----------+----------------+------------------------+------------------+



In [79]:
# Calcular a mediana dos salários por departamento
salario_dep_mediana = df_cleaned.groupBy('Area').agg(expr('percentile_approx(Salario, 0.5)').alias('Mediana'))
salario_dep_mediana.show()

+----------+-------+
|      Area|Mediana|
+----------+-------+
|financeiro| 5500.0|
| marketing| 4800.0|
|        rh| 4800.0|
| finaceiro| 6100.0|
+----------+-------+



In [75]:
# 5. Análise de Outliers: Identificar salários que estão fora do padrão (outliers) para cada departamento.
#. Calcular o primeiro quartil (Q1) e o terceiro quartil (Q3) para os salários por departamento
df_quartiles = df_cleaned.groupBy("Area").agg(
        expr("percentile_approx(Salario, 0.25)").alias("Q1"),
        expr("percentile_approx(Salario, 0.75)").alias("Q3")
    )

##. Calcular o IQR (Q3 - Q1)
df_quartiles = df_quartiles.withColumn("IQR", col("Q3") - col("Q1"))

###. Determinar os limites inferior e superior para detectar outliers
df_quartiles = df_quartiles.withColumn("Lower_Bound", col("Q1") - 1.5 * col("IQR")) \
    .withColumn("Upper_Bound", col("Q3") + 1.5 * col("IQR"))

####. Juntar os limites calculados com o dataframe original para identificar os outliers
df_with_outliers = df_cleaned.join(df_quartiles, on="Area")

#####. Filtrar os outliers (salários fora dos limites inferior e superior)
df_outliers = df_with_outliers.filter(
    (col("Salario") < col("Lower_Bound")) | (col("Salario") > col("Upper_Bound"))
)

###.###.Mostrar os outliers identificados
df_outliers.select("Area", "Salario", "Lower_Bound", "Upper_Bound").show()


+----+-------+-----------+-----------+
|Area|Salario|Lower_Bound|Upper_Bound|
+----+-------+-----------+-----------+
+----+-------+-----------+-----------+

